In [1]:
from geoalchemy2 import Geometry, load_spatialite
from flask_caching import Cache
from sqlalchemy import event, Table, insert, func, select, text
from sqlalchemy.orm import sessionmaker
import os
from flask import Flask
from quickannotator.db import Project, Image, AnnotationClass, Notification, Tile, Setting, Annotation
from quickannotator.db import db
import large_image
import math
import numpy as np 
from shapely.geometry import Point, Polygon
import random
from shapely.affinity import translate
import json
from tqdm import tqdm
import shapely
from shapely.geometry import shape
from quickannotator.api.v1.tile.helper import tiles_within_bbox
from quickannotator.api.v1.annotation.helper import count_annotations_within_bbox, delete_all_annotations
from quickannotator.api.v1.tile.helper import reset_all_tiles_seen
import geojson

/opt/QuickAnnotator/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-13 15:02:12,467	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
app = Flask("app")
SearchCache = Cache(config={'CACHE_TYPE': 'SimpleCache'})
SearchCache.init_app(app)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///quickannotator.db'
os.environ['SPATIALITE_LIBRARY_PATH'] = '/usr/lib/x86_64-linux-gnu/mod_spatialite.so'

db.app = app
db.init_app(app)


with app.app_context():
    db.engine.echo = False
    event.listen(db.engine, 'connect', load_spatialite)

In [7]:
def insert_project(app, db, name, description, is_dataset_large):
    with app.app_context():
        project = Project(name=name,
                          description=description,
                          is_dataset_large=is_dataset_large)
        db.session.add(project)
        db.session.commit()
        
def insert_image(app, db, project_id, name, path, height, width, dz_tilesize, embedding_coord, group_id, split):
    with app.app_context():
        image = Image(project_id=project_id,
                      name=name,
                      path=path,
                      height=height,
                      width=width,
                      dz_tilesize=dz_tilesize,
                      embedding_coord=embedding_coord,
                      group_id=group_id,
                      split=split)
        db.session.add(image)
        db.session.commit()

def insert_image_by_path(app, db, project_id, full_path):
    path = full_path.split("quickannotator/")[1]
    slide = large_image.getTileSource(path)
    name = os.path.basename(full_path)
    
    with app.app_context():
        image = Image(project_id=project_id,
                      name=name,
                      path=path,
                      height=slide.sizeY,
                      width=slide.sizeX,
                      dz_tilesize=slide.tileWidth,
                      embedding_coord="POINT (1 1)",
                      group_id=0,
                      split=0
                      )
        
        db.session.add(image)
        db.session.commit()
    
def insert_annotation_class(app, db, project_id, name, color, magnification, patchsize, tilesize, dl_model_objectref):
    with app.app_context():
        annotation_class = AnnotationClass(project_id=project_id,
                                           name=name,
                                           color=color,
                                           magnification=magnification,
                                           patchsize=patchsize,
                                           tilesize=tilesize,
                                           dl_model_objectref=dl_model_objectref)
        db.session.add(annotation_class)
        db.session.commit()
              
def insert_tile(app, db, image_id, annotation_class_id, geom, seen):
    with app.app_context():
        tile = Tile(image_id=image_id,
                    annotation_class_id=annotation_class_id,
                    geom=geom,
                    seen=seen)
        
        db.session.add(tile)
        db.session.commit()
        
def insert_tiles(app, db, image_id, annotation_class_id, seen):
    image_width, image_height = get_image_dimensions(app, image_id)
    
    with app.app_context():
        tile_size = AnnotationClass.query.filter_by(id=annotation_class_id).first().tilesize
    
    n_cols = math.ceil(image_width / tile_size)
    n_rows = math.ceil(image_height / tile_size)
    
    tiles = []
    for i in range(n_rows):
        for j in range(n_cols):
            t = Tile(image_id=image_id,
                     annotation_class_id=annotation_class_id,
                     geom=f"POLYGON(({j*tile_size} {i*tile_size}, {j*tile_size} {(i+1)*tile_size}, {(j+1)*tile_size} {(i+1)*tile_size}, {(j+1)*tile_size} {i*tile_size}, {j*tile_size} {i*tile_size}))",
                     seen=seen
                     )
            tiles.append(t)
    with app.app_context():
        db.session.add_all(tiles)
        db.session.commit()
        

def annotations_within_bbox(app, db, table, x1, y1, x2, y2):
    envelope = func.BuildMbr(x1, y1, x2, y2)
    # Right now we are selecting by centroid and not polygon.
    with app.app_context():
        stmt = table.select().where(func.ST_Intersects(table.c.centroid, envelope))
        result = db.session.execute(stmt).fetchall()
        
    return result
  
def get_image_dimensions(app, image_id):
    with app.app_context():
        image = Image.query.filter_by(id=image_id).first()
        return image.width, image.height
        
def create_annotation_table(db, image_id, annotation_class_id, gtpred):
    table_name = f"{image_id}_{annotation_class_id}_{gtpred}_annotation"
    table = Annotation.__table__.to_metadata(db.metadata, name=table_name)
    db.metadata.create_all(bind=db.engine, tables=[table])
            
def generate_random_polygon(max_area=10, centroid=(0, 0)):
    num_points = random.randint(10, 20)  # Polygons need at least 3 points

    # Generate points in polar coordinates
    radii = np.sqrt(np.random.uniform(0, 1, num_points))  # Square root ensures uniform distribution
    angles = np.linspace(0, 2 * np.pi, num_points, endpoint=False)

    # Convert polar coordinates to Cartesian coordinates
    points = [(r * np.cos(a), r * np.sin(a)) for r, a in zip(radii, angles)]

    # Sort points to form a simple polygon
    sorted_points = sorted(points, key=lambda p: np.arctan2(p[1], p[0]))

    # Create a Polygon and scale its area
    polygon = Polygon(sorted_points)

    # Calculate scaling factor
    current_area = polygon.area
    if current_area > 0:
        scaling_factor = np.sqrt(max_area / current_area)
        scaled_points = [(x * scaling_factor, y * scaling_factor) for x, y in sorted_points]
        polygon = Polygon(scaled_points)

    polygon = translate(polygon, xoff=centroid[0], yoff=centroid[1])

    return polygon
    
def generate_annotations(image_width, image_height, n_polygons, image_id, annotation_class_id, gtpred):
    annotations = []
    for i in range(n_polygons):
        c = Point(np.random.randint(image_width), np.random.randint(image_height))
        
        poly: Polygon = generate_random_polygon(max_area=10000, centroid=(c.x, c.y))
        d = {
            "image_id": image_id,
            "annotation_class_id": annotation_class_id,
            "isgt": gtpred == "gt",
            "centroid": poly.centroid.wkt,  # Adding SRID=0 for pixel-based coordinates
            "area": poly.area,  # The area of the polygon
            "polygon": poly.wkt,  # Adding SRID=0 for the polygon
            "custom_metrics": json.dumps({"iou": 0.5})  # Convert custom_metrics to a JSON string
        }
        annotations.append(d)
        
    return annotations

def extract_annotations_from_geojson_file(filepath):
    image_id = 1
    annotation_class_id = 2
    gtpred = "gt"

    path = filepath.split("quickannotator/")[1]
    with open(path, 'r') as file:
        # Load the JSON data into a Python dictionary
        data = json.load(file)
    
    annotations = []
    for i, d in enumerate(tqdm(data)):
        shapely_geometry = shape(d['geometry'])
        annotation = {
            "image_id": image_id,
            "annotation_class_id": annotation_class_id,
            "isgt": gtpred == "gt",
            "centroid": shapely_geometry.centroid.wkt,
            "area": shapely_geometry.area,
            "polygon": shapely_geometry.wkt,
            "custom_metrics": json.dumps({"iou": 0.5}) 
        }
        
        annotations.append(annotation)
        
    return annotations

        
def insert_existing_annotations(app, db, image_id, annotation_class_id, gtpred, filepath):
    path = filepath.split("quickannotator/")[1]
    with open(path, 'r') as file:
        # Load the JSON data into a Python dictionary
        data = json.load(file)
    all_anno = []
    
    table_name = f"{image_id}_{annotation_class_id}_{gtpred}_annotation"
    
    
    for i, d in enumerate(tqdm(data)):
        if d['properties']['classification']['name'] == 'tubule':
            shapely_geometry = shape(d['geometry'])
            annotation = {
                "image_id": image_id,
                "annotation_class_id": annotation_class_id,
                "isgt": gtpred == "gt",
                "centroid": shapely_geometry.centroid.wkt,
                "area": shapely_geometry.area,
                "polygon": shapely_geometry.wkt,
                "custom_metrics": json.dumps({"iou": 0.5}) 
            }
            
            all_anno.append(annotation)
        if len(all_anno)==1_000:
            with app.app_context():
                table = Table(table_name, db.metadata, autoload_with=db.engine)
                stmt = insert(table).values(all_anno)
                db.session.execute(stmt)
                db.session.commit()
            all_anno = []
    
    # commit any remaining annotations
    with app.app_context():
        table = Table(table_name, db.metadata, autoload_with=db.engine)
        stmt = insert(table).values(all_anno)
        db.session.execute(stmt)
        db.session.commit()
        
def insert_generated_annotations(app, db, image_id, annotation_class_id, gtpred, n):
    all_anno = []
    
    table_name = f"{image_id}_{annotation_class_id}_{gtpred}_annotation"
    with app.app_context():
        image = db.session.query(Image).filter_by(id=image_id).first()
        table = Table(table_name, db.metadata, autoload_with=db.engine)
        for i, d in enumerate(tqdm(range(n))):
            annotation = generate_annotations(image.width, image.height, 1, image_id, annotation_class_id, gtpred)[0]
            all_anno.append(annotation)
            if len(all_anno)==1_000:
                stmt = insert(table).values(all_anno)
                db.session.execute(stmt)
                db.session.commit()
                all_anno = []
    
    # commit any remaining annotations
    if len(all_anno) > 0:
        with app.app_context():
            table = Table(table_name, db.metadata, autoload_with=db.engine)
            stmt = insert(table).values(all_anno)
            db.session.execute(stmt)
            db.session.commit()
            
def insert_qupath_geojson_file(app, db, image_id, annotation_class_id, gtpred, filepath):
    '''
    This is expected to be a geojson feature collection file, with each polygon being a feature.
    
    '''
    path = filepath.split("quickannotator/")[1]
    with open(path, 'r') as file:
        # Load the JSON data into a Python dictionary
        data = json.load(file)["features"]
    all_anno = []
    
    table_name = f"{image_id}_{annotation_class_id}_{gtpred}_annotation"
    
    
    for i, d in enumerate(tqdm(data)):

        shapely_geometry = shape(d['geometry'])
        annotation = {
            "image_id": image_id,
            "annotation_class_id": annotation_class_id,
            "isgt": gtpred == "gt",
            "centroid": shapely_geometry.centroid.wkt,
            "area": shapely_geometry.area,
            "polygon": shapely_geometry.wkt,
            "custom_metrics": json.dumps({"iou": 0.5}) 
        }
        
        all_anno.append(annotation)
        
        if len(all_anno)==1_000:
            with app.app_context():
                table = Table(table_name, db.metadata, autoload_with=db.engine)
                stmt = insert(table).values(all_anno)
                db.session.execute(stmt)
                db.session.commit()
            all_anno = []
    
    # commit any remaining annotations
    with app.app_context():
        table = Table(table_name, db.metadata, autoload_with=db.engine)
        stmt = insert(table).values(all_anno)
        db.session.execute(stmt)
        db.session.commit()


In [8]:
models = [Project, Image, AnnotationClass, Notification, Tile, Setting]
with app.app_context():
    db.metadata.create_all(bind=db.engine, tables=[item.__table__ for item in models])
    create_annotation_table(db, 1, 1, "gt")
    create_annotation_table(db, 1, 2, "gt")
    create_annotation_table(db, 1, 3, "gt")
    create_annotation_table(db, 2, 1, "gt")
    create_annotation_table(db, 2, 2, "gt")
    create_annotation_table(db, 2, 3, "gt")

    create_annotation_table(db, 1, 2, "pred")
    create_annotation_table(db, 1, 3, "pred")
    create_annotation_table(db, 2, 2, "pred")
    create_annotation_table(db, 2, 3, "pred")
    

/tmp/ipykernel_71267/1324745128.py:104: SAWarning: Table 'annotation' already exists within the given MetaData - not copying.
  table = Annotation.__table__.to_metadata(db.metadata, name=table_name)


In [3]:
with app.app_context():
    reset_all_tiles_seen(db.session)
    delete_all_annotations(db.session, 1, 2, False)

In [ ]:
insert_project(app, db, 
               name="example_project", 
               description="test", 
               is_dataset_large=False)
insert_image_by_path(app, db,
                     project_id=1,
                     full_path="quickannotator/data/test_ndpi/13_266069_040_003 L02 PAS.ndpi"
                     )

insert_image_by_path(app, db,
                     project_id=1,
                     full_path="quickannotator/data/test_ndpi/TCGA-23-2072-01Z-00-DX1.478243FF-BFF0-48A4-ADEA-DE789331A50E.svs")

insert_annotation_class(app, db,
                        project_id=None,
                        name="Tissue Mask",
                        color="black",
                        magnification=None,
                        patchsize=None,
                        tilesize=None,
                        dl_model_objectref=None)

insert_annotation_class(app, db,
                        project_id=1,
                        name="Tubule",
                        color="red",
                        magnification=10,
                        patchsize=256,
                        tilesize=2048,
                        dl_model_objectref=None)

insert_annotation_class(app, db,
                        project_id=1,
                        name="Lumen",
                        color="blue",
                        magnification=10,
                        patchsize=256,
                        tilesize=4096,
                        dl_model_objectref=None)

insert_tiles(app, db, image_id=1, annotation_class_id=2, seen=0)

insert_tiles(app, db, image_id=2, annotation_class_id=2, seen=0)

insert_tiles(app, db, image_id=1, annotation_class_id=3, seen=0)

insert_tiles(app, db, image_id=2, annotation_class_id=3, seen=0)

insert_generated_annotations(app, db, image_id=2, annotation_class_id=2, gtpred="gt", n=1_000_000)
insert_generated_annotations(app, db, image_id=2, annotation_class_id=3, gtpred="gt", n=100_000)

In [4]:
insert_existing_annotations(app, db,
                            image_id=1,
                            annotation_class_id=2,
                            gtpred="gt",
                            filepath='quickannotator/data/test_ndpi/13_266069_040_003 L02 PAS.json'
                            )

100%|██████████| 88605/88605 [00:06<00:00, 13469.13it/s]


In [ ]:

insert_qupath_geojson_file(app, db,
                            image_id=1,
                            annotation_class_id=1,
                            gtpred='gt',
                            filepath='quickannotator/data/test_ndpi/13_266069_040_003 L02 PAS_tissue_mask.geojson')


In [ ]:

insert_qupath_geojson_file(app, db, 
                           image_id=2,
                           annotation_class_id=1,
                           gtpred='gt',
                           filepath='quickannotator/data/test_ndpi/TCGA-23-2072-01Z-00-DX1.478243FF-BFF0-48A4-ADEA-DE789331A50E_tissue_mask.geojson')


In [ ]:
with app.app_context():
    table_name = '2_2_gt_annotation'
    table = Table(table_name, db.metadata, autoload_with=db.engine)
    %timeit anns = annotations_within_bbox(app, db, table, 10000,10000,10150,10150)

In [ ]:
from quickannotator.api.v1.annotation.helper import annotations_within_bbox_spatial

In [ ]:
with app.app_context():
    result = annotations_within_bbox_spatial('1_2_gt_annotation', 40000, 40000, 45000, 45000)

In [ ]:
result

In [ ]:
from quickannotator.api.v1.annotation import AnnRespSchema

In [ ]:
result[0].centroid